**Mine-Level Temporal Aggregation**

Assumptions (important)

objects_gdf schema includes:

date            # detection date (datetime)
start_date      # first appearance of this object
area_ha         # object area in hectares
geometry        # already within legal boundary


Objects persist once created (no disappearance unless you explicitly model recovery)

Recovery is optional and usually zero (we’ll code it safely)

GROUP A — Prepare & Sanity-Check Inputs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# import spacial object formation pipeline
from ipynb.fs.defs.space_object_form_pipeline import spatial_objects_formation , spatial_objects_degrees

In [ ]:
# Ensure datetime
objects_gdf["date"] = pd.to_datetime(objects_gdf["date"])
objects_gdf["start_date"] = pd.to_datetime(objects_gdf["start_date"])

# Sorted unique monitoring dates
dates = sorted(objects_gdf["date"].unique())


GROUP B — Total Excavated Area per Date

Definition (locked)

[
A_{\text{mine}}(t) = \sum_{\text{objects present at } t} \text{area}_{ha}
]

In [ ]:
total_excavated_area = []

for t in dates:
    area_t = objects_gdf.loc[
        objects_gdf["start_date"] <= t, "area_ha"
    ].sum()
    total_excavated_area.append(area_t)

total_excavated_area = pd.Series(
    total_excavated_area, index=dates, name="total_excavated_area_ha"
)


GROUP C — New Excavation per Date

In [ ]:
new_excavation = (
    objects_gdf
    .groupby("start_date")["area_ha"]
    .sum()
    .reindex(dates, fill_value=0.0)
)

new_excavation.name = "new_excavation_ha"


GROUP D — Recovery (Handled Explicitly & Safely)

In [ ]:
recovered_area = pd.Series(
    0.0, index=dates, name="recovered_ha"
)


GROUP E — Build Final Mine-Level Time Series

In [ ]:
mine_timeseries = pd.DataFrame({
    "date": dates,
    "total_excavated_area_ha": total_excavated_area.values,
    "new_excavation_ha": new_excavation.values,
    "recovered_ha": recovered_area.values
})


GROUP F — Net Change & Rate

mine_timeseries["net_change_ha"] = (
    mine_timeseries["total_excavated_area_ha"]
    .diff()
    .fillna(0.0)
)


In [ ]:
mine_timeseries["excavation_rate_ha_per_step"] = (
    mine_timeseries["new_excavation_ha"]
)


GROUP G — Optional Temporal Smoothing 

In [ ]:
mine_timeseries["total_excavated_area_smooth"] = (
    mine_timeseries["total_excavated_area_ha"]
    .rolling(window=3, center=True, min_periods=1)
    .median()
)


# GROUP H — Visualization (MANDATORY DELIVERABLE)

In [ ]:
# Total excavated area over time

plt.figure(figsize=(9,4))
plt.plot(
    mine_timeseries["date"],
    mine_timeseries["total_excavated_area_ha"],
    marker="o",
    label="Total Excavated Area"
)
plt.plot(
    mine_timeseries["date"],
    mine_timeseries["total_excavated_area_smooth"],
    linewidth=2,
    label="Smoothed"
)
plt.xlabel("Date")
plt.ylabel("Area (ha)")
plt.title("Total Excavated Area Within Legal Mine Boundary")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# New excavation per date (growth)

plt.figure(figsize=(9,4))
plt.bar(
    mine_timeseries["date"],
    mine_timeseries["new_excavation_ha"],
    width=10
)
plt.xlabel("Date")
plt.ylabel("New Excavation (ha)")
plt.title("New Excavation Detected Per Date")
plt.grid(True)
plt.show()


GROUP I — Export for Reporting

In [ ]:
mine_timeseries.to_csv(
    "mine_level_excavation_timeseries.csv",
    index=False
)
